In [1]:
import pyanitools as pyt
import pyNeuroChem as pyc
import hdnntools as hdt
import numpy as np

from ase_interface import ANIENS
from ase_interface import ensemblemolecule
from ase.atoms import Atoms

import os

import matplotlib.pyplot as plt

/home/jujuman/.conda/envs/my-rdkit-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


van Der Waals correction will be unavailable. Please install ased3


In [2]:
h5test = '/home/jujuman/Scratch/Research/tin_research/metal_data/U12.0_test.h5'

ens_path = '/home/jujuman/Scratch/Research/tin_research/metal_data/models/gutz_lc_models/'

cns = '/train0/rX-2.8R_32-2.0A_a8-8.params'
sae = '/train0/sae.dat'
nnf = '/train'
Nn = 8

In [8]:
model_paths = sorted(os.listdir(ens_path))

comp_all = {}

for model_path in model_paths:
    wkpath = ens_path+model_path
    print(wkpath)
    aens = ensemblemolecule(wkpath + cns,wkpath + sae,wkpath + nnf,Nn)
    
    comp = {'E':([],[]),
            'F':([],[]),}
    
    adl = pyt.anidataloader(h5test)
    
    for data in adl:
        print(data['path'])
        X = data['coordinates']
        S = data['species']
        E = data['energies']
        F = data['forces']
        C = data['cell']
        
        for x,e,f,c in zip(X,E,F,C):
            comp['E'][0].append(e)
            comp['F'][0].append(f.flatten())
            
            aens.set_pbc(True,True,True)
            celi = (np.linalg.inv(C)).astype(np.float32)
            aens.set_cell((C).astype(np.float32), celi)

            aens.set_molecule(X=x,S=S)
            ea,fa,es,fs = aens.compute_mean_props()

            comp['E'][1].append(ea)
            comp['F'][1].append(fa.flatten())
            
    comp['E'][0] = np.concatenate(comp['E'][0])
    comp['E'][1] = np.concatenate(comp['E'][1])
    comp['F'][0] = np.concatenate(comp['F'][0])
    comp['F'][1] = np.concatenate(comp['F'][1])
    comp_all[model_path] = comp

/home/jujuman/Scratch/Research/tin_research/metal_data/models/gutz_lc_models/U12.0_002p
/U12.0/kT0.075
/U12.0/kT0.15


KeyboardInterrupt: 